### Assignment 2 Part B<br>

Name: Manmeet Singh<br>
Id: 30749476

In [ ]:
# import statements
from time import sleep
from kafka import KafkaConsumer
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
from json import loads
import pandas as pd
from pyspark.sql import functions as F
import time
# this line is needed for the inline display of graphs in Jupyter Notebook
%matplotlib notebook

topic = 'memory_producer'

def connect_kafka_consumer():
    _consumer = None
    try:
         _consumer = KafkaConsumer(topic,
                                   consumer_timeout_ms=20000, # stop iteration if no message after 10 sec
                                   auto_offset_reset='latest', # comment this if you don't want to consume earliest available message
                                   bootstrap_servers=['localhost:9092'],
                                   value_deserializer=lambda x: loads(x.decode('ascii')),
                                   api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _consumer

def init_plots():
    try:
        width = 10
        height = 6
        fig = plt.figure(figsize=(width,height)) # create new figure
        ax = fig.add_subplot(111) # adding the subplot axes to the given grid position
        fig.suptitle('Real-time uniform stream data visualization') # giving figure a title
        ax.set_xlabel('Time')
        ax.set_ylabel('Count')        
        fig.show() # displaying the figure
        fig.canvas.draw() # drawing on the canvas
        return fig, ax
    except Exception as ex:
        print(str(ex))
        
def consume_messages(consumer,fig,ax):
    try:
        time = []
        M_4, M_5, M_6, M_7, M_8 = [],[],[],[],[]
        machine_records = {4:0,5:0,6:0,7:0,8:0}
        i = 0
        for message in consumer:   
            ts=""
            data_received = list(message.value)
            for data in data_received:
                machine = data['machine']
                machine_records[machine] += 1
                ts = data['ts']
            M_4.append(machine_records[4])
            M_5.append(machine_records[5])
            M_6.append(machine_records[6])
            M_7.append(machine_records[7])
            M_8.append(machine_records[8])
            time.append(ts)
            i+=1
            if i==60:
                ax.clear()
                ax.plot(time,M_4, label='Machine 4')
                ax.plot(time,M_5, label='Machine 5')
                ax.plot(time,M_6, label='Machine 6')
                ax.plot(time,M_7, label='Machine 7')
                ax.plot(time,M_8, label='Machine 8')
                ax.legend()
                fig.canvas.draw()               
                time.pop(0) 
                M_4.pop(0)
                M_5.pop(0)
                M_6.pop(0)
                M_7.pop(0)
                M_8.pop(0)
                i=0
        plt.close('all')
    except Exception as ex:
        print(str(ex))

In [ ]:
%matplotlib notebook
consumer = connect_kafka_consumer()
fig, ax = init_plots()
consume_messages(consumer,fig,ax) 